In [1]:
import sqlalchemy
import pandas as pd
import numpy as np
import sql_functions as sf

from sql_functions import *
import psycopg2

#import matplotlib as mpl
#import matplotlib.pyplot as plt

#import seaborn as sns

In [2]:
schema = 'capstone_jmrs'
spotahome_final_3 = sf.get_dataframe(f'SELECT * FROM {schema}.spotahome_final_3')
spotahome_final_3

,platform_id,platform,neighborhood,property_type,housing_type,price_pcm,title,furnished,bathrooms,m2,bedrooms,available_from_total
0,642015,spotahome,Kensington and Chelsea,studios,Studio,3915,"Studio apartment for rent in South Kensington,...",furnished,NaN,NaN,NaN,2022-10-06
1,607188,spotahome,Kensington and Chelsea,studios,Studio,3033,"Studio for rent in Earls Court, London",furnished,NaN,NaN,NaN,2022-10-05
2,643269,spotahome,Kensington and Chelsea,studios,Studio,3915,"Studio apartment for rent in Notting Hill, London",furnished,NaN,NaN,NaN,2022-10-12
3,574861,spotahome,Kensington and Chelsea,studios,Studio,3033,"Studio for rent in Earls Court, London",furnished,NaN,NaN,NaN,2022-10-05
4,642044,spotahome,Kensington and Chelsea,studios,Studio,4407,"Studio apartment for rent in South Kensington,...",furnished,NaN,NaN,NaN,2022-11-01
...,...,...,...,...,...,...,...,...,...,...,...,...
831,844665,spotahome,Hackney,apartments/bedrooms:3more,Apartment,8250,4-bedroom apartment for rent in London,furnished,2.0,NaN,4.0,2023-01-08
832,843245,spotahome,Hackney,apartments/bedrooms:3more,Apartment,9000,4-bedroom apartment for rent in London,furnished,2.0,NaN,4.0,2023-01-11
833,398966,spotahome,Hackney,apartments/bedrooms:3more,Apartment,2100,4-bedroom apartment in Hackney,furnished,2.0,85.0,5.0,2024-01-06
834,602243,spotahome,Hackney,apartments/bedrooms:3more,Apartment,9000,4 bedrooms warehouse conversion for rent in Ha...,furnished,2.0,NaN,4.0,2023-08-04


In [3]:
spotahome_final_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 836 entries, 0 to 835
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   platform_id           836 non-null    int64  
 1   platform              836 non-null    object 
 2   neighborhood          836 non-null    object 
 3   property_type         836 non-null    object 
 4   housing_type          836 non-null    object 
 5   price_pcm             836 non-null    int64  
 6   title                 836 non-null    object 
 7   furnished             836 non-null    object 
 8   bathrooms             524 non-null    float64
 9   m2                    270 non-null    float64
 10  bedrooms              592 non-null    float64
 11  available_from_total  836 non-null    object 
dtypes: float64(3), int64(2), object(7)
memory usage: 78.5+ KB


In [4]:
spotahome_final_3.describe()

,platform_id,price_pcm,bathrooms,m2,bedrooms
count,836.000000,836.000000,524.000000,270.000000,592.000000
mean,544861.363636,4395.705742,1.322519,62.888889,1.773649
std,158406.159183,1947.398751,0.640408,67.230632,0.897137
min,98665.000000,1112.000000,1.000000,1.000000,1.000000
25%,422244.500000,3038.250000,1.000000,40.000000,1.000000
50%,568773.000000,4125.000000,1.000000,55.000000,2.000000
75%,642051.250000,5205.000000,2.000000,75.000000,2.000000
max,867016.000000,13892.000000,5.000000,1000.000000,6.000000


In [5]:
spotahome_final_3.corr()

,platform_id,price_pcm,bathrooms,m2,bedrooms
platform_id,1.000000,0.181987,0.146372,0.064962,0.177743
price_pcm,0.181987,1.000000,0.509969,0.171763,0.437569
bathrooms,0.146372,0.509969,1.000000,0.345914,0.626310
m2,0.064962,0.171763,0.345914,1.000000,0.362784
bedrooms,0.177743,0.437569,0.626310,0.362784,1.000000


In [6]:
spotahome_final_3.eval('price_per_sqm = price_pcm / m2', inplace=True)

In [7]:
spotahome_final_3

,platform_id,platform,neighborhood,property_type,housing_type,price_pcm,title,furnished,bathrooms,m2,bedrooms,available_from_total,price_per_sqm
0,642015,spotahome,Kensington and Chelsea,studios,Studio,3915,"Studio apartment for rent in South Kensington,...",furnished,NaN,NaN,NaN,2022-10-06,NaN
1,607188,spotahome,Kensington and Chelsea,studios,Studio,3033,"Studio for rent in Earls Court, London",furnished,NaN,NaN,NaN,2022-10-05,NaN
2,643269,spotahome,Kensington and Chelsea,studios,Studio,3915,"Studio apartment for rent in Notting Hill, London",furnished,NaN,NaN,NaN,2022-10-12,NaN
3,574861,spotahome,Kensington and Chelsea,studios,Studio,3033,"Studio for rent in Earls Court, London",furnished,NaN,NaN,NaN,2022-10-05,NaN
4,642044,spotahome,Kensington and Chelsea,studios,Studio,4407,"Studio apartment for rent in South Kensington,...",furnished,NaN,NaN,NaN,2022-11-01,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
831,844665,spotahome,Hackney,apartments/bedrooms:3more,Apartment,8250,4-bedroom apartment for rent in London,furnished,2.0,NaN,4.0,2023-01-08,NaN
832,843245,spotahome,Hackney,apartments/bedrooms:3more,Apartment,9000,4-bedroom apartment for rent in London,furnished,2.0,NaN,4.0,2023-01-11,NaN
833,398966,spotahome,Hackney,apartments/bedrooms:3more,Apartment,2100,4-bedroom apartment in Hackney,furnished,2.0,85.0,5.0,2024-01-06,24.705882
834,602243,spotahome,Hackney,apartments/bedrooms:3more,Apartment,9000,4 bedrooms warehouse conversion for rent in Ha...,furnished,2.0,NaN,4.0,2023-08-04,NaN


In [8]:
spotahome_final_3['price_per_sqm'] = spotahome_final_3['price_per_sqm'].round(decimals = 2)

In [9]:
spotahome_final_3

,platform_id,platform,neighborhood,property_type,housing_type,price_pcm,title,furnished,bathrooms,m2,bedrooms,available_from_total,price_per_sqm
0,642015,spotahome,Kensington and Chelsea,studios,Studio,3915,"Studio apartment for rent in South Kensington,...",furnished,NaN,NaN,NaN,2022-10-06,NaN
1,607188,spotahome,Kensington and Chelsea,studios,Studio,3033,"Studio for rent in Earls Court, London",furnished,NaN,NaN,NaN,2022-10-05,NaN
2,643269,spotahome,Kensington and Chelsea,studios,Studio,3915,"Studio apartment for rent in Notting Hill, London",furnished,NaN,NaN,NaN,2022-10-12,NaN
3,574861,spotahome,Kensington and Chelsea,studios,Studio,3033,"Studio for rent in Earls Court, London",furnished,NaN,NaN,NaN,2022-10-05,NaN
4,642044,spotahome,Kensington and Chelsea,studios,Studio,4407,"Studio apartment for rent in South Kensington,...",furnished,NaN,NaN,NaN,2022-11-01,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
831,844665,spotahome,Hackney,apartments/bedrooms:3more,Apartment,8250,4-bedroom apartment for rent in London,furnished,2.0,NaN,4.0,2023-01-08,NaN
832,843245,spotahome,Hackney,apartments/bedrooms:3more,Apartment,9000,4-bedroom apartment for rent in London,furnished,2.0,NaN,4.0,2023-01-11,NaN
833,398966,spotahome,Hackney,apartments/bedrooms:3more,Apartment,2100,4-bedroom apartment in Hackney,furnished,2.0,85.0,5.0,2024-01-06,24.71
834,602243,spotahome,Hackney,apartments/bedrooms:3more,Apartment,9000,4 bedrooms warehouse conversion for rent in Ha...,furnished,2.0,NaN,4.0,2023-08-04,NaN


In [10]:
spotahome_clean = spotahome_final_3

In [11]:
# call the schema created for this project
schema = 'capstone_jmrs'
# get the function to connect to the database
engine = get_engine()

# give the table a unique name
table_name = 'spotahome_clean'

# import the table to sql
if engine != None:
    try:
        spotahome_clean.to_sql(name=table_name,
                                   con=engine,
                                   if_exists='replace',
                                   schema=schema,
                                   index=False,
                                   chunksize=5000,
                                   method='multi')
        print(f"The {table_name} table was imported successfully.")

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None


The spotahome_clean table was imported successfully.
